# Werkcollege-opdrachten Week 1.3

## Dependencies importeren

Kopieer in het codeblok hieronder van het vorige werkcollege de import-code voor de dependencies die het vaakst worden gebruikt om data in te lezen. Geef er ook de gebruikelijke aliassen aan.<br>
Zet eventuele warnings uit.

In [146]:
import pandas as pd
import sqlite3
import warnings
import numpy as np
warnings.simplefilter('ignore')

Zet de volgende bestanden in een makkelijk te vinden map, als je dat nog niet gedaan hebt:
- go_sales.sqlite
- go_crm.sqlite
- go_staff.sqlite

## Databasetabellen inlezen

Kopieer in het codeblok hieronder van het vorige werkcollege de code om een connectie met het bestand go_sales.sqlite te maken.

In [ ]:
conn = sqlite3.connect('go_sales.sqlite')

Lees van de ingelezen go_sales-database te volgende tabellen in met behulp van "SELECT * FROM *tabel*".
- product
- product_type
- product_line
- sales_staff
- sales_branch
- retailer_site
- country
- order_header
- order_details
- returned_item
- return_reason

In [ ]:
products = pd.read_sql_query("SELECT * FROM product", conn)
product_types = pd.read_sql_query("SELECT * FROM product_type", conn)
product_lines = pd.read_sql_query("SELECT * FROM product_line", conn)
sales_staffs = pd.read_sql_query("SELECT * FROM sales_staff", conn)
sales_branches = pd.read_sql_query("SELECT * FROM sales_branch", conn)
retailer_sites = pd.read_sql_query("SELECT * FROM retailer_site", conn)
countries = pd.read_sql_query("SELECT * FROM country", conn)
order_headers = pd.read_sql_query("SELECT * FROM order_header", conn)
order_details =pd.read_sql_query("SELECT * FROM order_details", conn)
returned_items = pd.read_sql_query("SELECT * FROM returned_item", conn)
return_reasons = pd.read_sql_query("SELECT * FROM return_reason", conn)



Krijg je een "no such table" error? Dan heb je misschien met .connect() per ongeluk een leeg  databasebestand (.sqlite) aangemaakt.<br> <u>Let op:</u> lees eventueel de informatie uit het vorige werkcollege nog eens goed door.

Als je tijdens onderstaande opdrachten uit het oog verliest welke tabellen er allemaal zijn, kan je deze Pythoncode uitvoeren:

In [ ]:
sql_query = "SELECT * FROM sqlite_master WHERE type='table';"
#Vul dit codeblok verder in
countries
#Op de puntjes hoort de connectie naar go_sales óf go_staff óf go_crm te staan.

Let op! Voor alle onderstaande opdrachten mag je <u>alleen Python</u> gebruiken, <u>geen SQL!</u>

## Selecties op één tabel zonder functies

Geef een overzicht met daarin de producten en hun productiekosten waarvan de productiekosten lager dan 100 dollar en hoger dan 50 dollar ligt. (2 kolommen, 23 rijen)

In [ ]:
productCostFilteredAbove = products['PRODUCTION_COST'] < 100
productCostFilteredBelow = products['PRODUCTION_COST'] > 50
products.loc[(productCostFilteredAbove & productCostFilteredBelow), ['PRODUCT_NUMBER', 'PRODUCTION_COST' ]]


Geef een overzicht met daarin de producten en hun marge waarvan de marge lager dan 20 % of hoger dan 60 % ligt. (2 kolommen, 7 rijen) 

In [ ]:
productCostFilteredAbove = products['MARGIN'] < 0.20
productCostFilteredBelow = products['MARGIN'] > 0.60
products.loc[(productCostFilteredAbove | productCostFilteredBelow), ['PRODUCT_NUMBER', 'MARGIN' ]]


Geef een overzicht met daarin de landen waar met francs wordt betaald. Sorteer de uitkomst op land.  (1 kolom, 3 rijen)

In [ ]:
francCountries = countries['CURRENCY_NAME'].str.contains('francs')
countries.loc[(francCountries), :]

Geef een overzicht met daarin de verschillende introductiedatums waarop producten met meer dan 50% marge worden geïntroduceerd (1 kolom, 7 rijen) 

In [ ]:

productDatumMargeFilter = products['MARGIN'] > 0.50
products.loc[(productDatumMargeFilter), 'INTRODUCTION_DATE'].drop_duplicates()



Geef een overzicht met daarin het eerste adres en de stad van verkoopafdelingen waarvan zowel het tweede adres als de regio bekend is (2 kolommen, 7 rijen)

In [ ]:
filterSalesBranch1 = sales_branches['ADDRESS2'].notna()
filterSalesBranch2 = sales_branches['REGION'].notna()
sales_branches.loc[(filterSalesBranch1 &filterSalesBranch2), ['ADDRESS1', 'CITY']]

Geef een overzicht met daarin de landen waar met dollars (dollars of new dollar) wordt betaald. Sorteer de uitkomst op land. (1 kolom, 4 rijen) 

In [ ]:
dollarCountries = countries['CURRENCY_NAME'].str.contains('dollar')
countries.loc[(dollarCountries), :].sort_values('COUNTRY')

Geef een overzicht met daarin beide adressen en de stad van vestigingen van klanten waarvan de postcode begint met een ‘D’ (van duitsland). Filter op vestigingen die een tweede adres hebben. (3 kolommen, 2 rijen) 

In [ ]:
germanRetailers = retailer_sites['POSTAL_ZONE'].str[0] == 'D'
address2Retailers = retailer_sites['ADDRESS2'].notna()
retailer_sites.loc[(germanRetailers & address2Retailers), ['ADDRESS1', 'ADDRESS2', 'CITY']]

## Selecties op één tabel met functies

Geef het totaal aantal producten dat is teruggebracht (1 waarde) 

In [ ]:
returned_items['RETURN_CODE'].nunique()

Geef het aantal regio’s waarin verkoopafdelingen gevestigd zijn. (1 waarde)

In [ ]:
sales_branches['REGION'].nunique()


Maak 3 variabelen:
- Een met de laagste
- Een met de hoogste
- Een met de gemiddelde (afgerond op 2 decimalen)

marge van producten (3 kolommen, 1 rij) 

In [ ]:
products[['MARGIN']].agg([max, min,np.mean]).round(2)

Geef het aantal vestigingen van klanten waarvan het 2e adres niet bekend is (1 waarde)

In [ ]:
retailer_sites.loc[(retailer_sites['ADDRESS2'].isna()), :]['RETAILER_SITE_CODE'].nunique()

Geef de gemiddelde kostprijs van de verkochte producten waarop korting (unit_sale_price < unit_price) is verleend (1 waarde) 

In [ ]:
order_detailsMetKorting = order_details['UNIT_SALE_PRICE'] < order_details['UNIT_PRICE']
order_details.loc[(order_detailsMetKorting), ['UNIT_COST']].agg(np.mean)

Geef een overzicht met daarin het aantal medewerkers per medewerkersfunctie (2 kolommen, 7 rijen) 

In [ ]:
sales_staffs.groupby('POSITION_EN')['SALES_STAFF_CODE'].count()

Geef een overzicht met daarin per telefoonnummer het aantal medewerkers dat op dat telefoonnummer bereikbaar is. Toon alleen de telefoonnummer waarop meer dan 4 medewerkers bereikbaar zijn. (2 kolommen, 10 rijen) 

In [ ]:
salesStaffPhoneFilter = sales_staffs.groupby('WORK_PHONE')['WORK_PHONE'].count() > 4


salesStaffPhoneFilter.loc[salesStaffPhoneFilter]


## Selecties op meerdere tabellen zonder functies

Geef een overzicht met daarin het eerste adres en de stad van vestigingen van klanten uit ‘Netherlands’ (2 kolommen, 20 rijen) 

In [ ]:
countriesAndRetailers = pd.merge(countries, retailer_sites, left_on= 'COUNTRY_CODE', how= 'inner', right_on='COUNTRY_CODE')
netherlandRetailersFilter = countriesAndRetailers['COUNTRY'] == 'Netherlands'
countriesAndRetailers.loc[(netherlandRetailersFilter), ['ADDRESS1', 'CITY']]

Geef een overzicht met daarin de productnamen die tot het producttype ‘Eyewear’ behoren. (1 kolom, 5 rijen) 

In [ ]:
productCodeProducts = pd.merge(product_types, products, left_on= 'PRODUCT_TYPE_CODE', how= 'inner', right_on='PRODUCT_TYPE_CODE')
eyewearProductsFilter = productCodeProducts['PRODUCT_TYPE_EN'] == 'Eyewear'
productCodeProducts.loc[(eyewearProductsFilter), 'PRODUCT_NAME']

Geef een overzicht met daarin alle unieke eerste adressen van klantvestigingen en de voornaam en achternaam van de verkopers die ‘Branch Manager’ zijn en aan deze vestigingen hebben verkocht (3 kolommen, 1 rij) 

In [164]:
merge1 = pd.merge(order_headers, retailer_sites, how= 'inner')
merge2 = pd.merge(merge1, sales_staffs, how='inner')
branchManagers = (merge2['POSITION_EN'] == 'Branch Manager')
merge2.loc[(branchManagers), ['ADDRESS1', 'FIRST_NAME', 'LAST_NAME']].drop_duplicates('ADDRESS1')

,ADDRESS1,FIRST_NAME,LAST_NAME
3185,"Alameda Santos, 9876",Bayard,Lopes


Geef een overzicht met daarin van de verkopers hun functie en indien zij iets hebben verkocht de datum waarop de verkoop heeft plaatsgevonden. Laat alleen de verschillende namen van de posities zien van de verkopers die het woord ‘Manager’ in hun positienaam hebben staan. (2 kolommen, 7 rijen) 

In [170]:
#merge3 = pd.merge(order_headers, sales_staffs, how= 'right')
#merge3Filter = merge3['POSITION_EN'].str.contains('Manager') 
#merge3.loc[(merge3Filter), :]

merge3 = pd.merge(order_headers, sales_staffs, how="inner", on="SALES_STAFF_CODE")
manager_rows = merge3["POSITION_EN"].str.contains("Manager")

result = (
    merge3.loc[manager_rows, ["POSITION_EN", "ORDER_DATE"]]
    .drop_duplicates(subset=["POSITION_EN"])
)

result

,POSITION_EN,ORDER_DATE
3185,Branch Manager,2024-09-09


Geef een overzicht met daarin de verschillende namen van producten en bijbehorende namen van producttypen van de producten waarvoor ooit meer dan 750 stuks tegelijk verkocht zijn. (2 kolommen, 9 rijen) 

Geef een overzicht met daarin de productnamen waarvan ooit meer dan 40% korting is verleend. De formule voor korting is: (unit_price - unit_sale_price) / unit_price (1 kolom, 8 rijen) 

Geef een overzicht met daarin de retourreden van producten waarvan ooit meer dan 90% van de aangeschafte hoeveelheid is teruggebracht (return_quantity/quantity). (1 kolom, 3 rijen) 

## Selecties op meerdere tabellen met functies

Geef een overzicht met daarin per producttype het aantal producten die tot dat producttype behoren. (2 kolommen, 21 rijen) 

Geef een overzicht met daarin per land het aantal vestigingen van klanten die zich in dat land bevinden. (2 kolommen, 21 rijen) 

Geef een overzicht met daarin van de producten behorend tot het producttype ‘Cooking Gear’ per productnaam de totaal verkochte hoeveelheid en de gemiddelde verkoopprijs. Sorteer de uitkomst op totaal verkochte hoeveelheid. (6 kolommen, 10 rijen)

Geef een overzicht met daarin per land de naam van het land, de naam van de stad waar de verkoopafdeling is gevestigd (noem de kolomnaam in het overzicht ‘verkoper’) en het aantal steden waar zich klanten bevinden in dat land (noem de kolomnaam in het overzicht ‘klanten’) (3 kolommen, 29 rijen) 

## Pythonvertalingen van SUBSELECT en UNION met o.a. for-loops

Geef een overzicht met daarin de voornaam en de achternaam van de medewerkers die nog nooit wat hebben verkocht (2 kolommen, 25 rijen) 

Geef een overzicht met daarin het aantal producten waarvan de marge lager is dan de gemiddelde marge van alle producten samen. Geef in het overzicht tevens aan wat de gemiddelde marge is van dit aantal producten waarvan de marge lager dan de gemiddelde marge van alle producten samen is. (1 kolom, 2 rijen) 

Geef een overzicht met daarin de namen van de producten die voor meer dan 500 (verkoopprijs) zijn verkocht maar nooit zijn teruggebracht. (1 kolom, 13 rijen) 

Geef een overzicht met daarin per (achternaam van) medewerker of hij/zij manager is of niet, door deze informatie toe te voegen als extra 'Ja/Nee'-kolom.<br>
Hint: gebruik een for-loop waarin je o.a. bepaalt of het woord 'Manager' in de functie (position_en) staat. (2 kolommen, 102 rijen).

Met de onderstaande code laat je Python het huidige jaar uitrekenen.

In [ ]:
from datetime import date
date.today().year

Met de onderstaande code selecteer je op een bepaald jaartal uit een datum.

In [ ]:
from datetime import datetime

date_str = '16-8-2013'
date_format = '%d-%m-%Y'
date_obj = datetime.strptime(date_str, date_format)

date_obj.year

Geef met behulp van bovenstaande hulpcode een overzicht met daarin op basis van het aantal jaar dat iemand in dienst is of een medewerker ‘kort in dienst’ (minder dan 30 jaar in dienst) of een ‘lang in dienst’ (groter dan gelijk aan 30 jaar in dienst) is. Geef daarbij per medewerker in een aparte kolom zowel ‘kort in dienst’ als ‘lang in dienst’ aan. Gebruik (wederom) een for-loop.<br>
(2 kolommen, 102 rijen) 